In [ ]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [1]:
import pandas as pd
import numpy as np


In [6]:
df=pd.read_csv('data/test_geometric_features_normalized.csv')
# df2=pd.read_csv('geometric_features.csv')

linearity             740
planarity             740
sphericity            740
omnivariance          740
anisotropy            740
eigenentropy          740
sum_of_eigenvalues    740
change_curvature      740
id                    740
class                  12
reflectance            19
dtype: int64
anisotropy            2438
change_curvature      2438
eigenentropy          2438
linearity             2438
omnivariance          2438
planarity             2438
sphericity            2438
sum_of_eigenvalues    2438
class                   47
reflectance             37
dtype: int64
